In [5]:
from __future__ import print_function
import torch
import torch.optim as optim
from torchvision import datasets, transforms

from model import Net
from utils import train, test
import torch
import torch.nn as nn
import torch.nn.functional as F


In [14]:

# Prepare for GPU-based training
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# Prepare the dataset and dataloaders
# Training data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
                        transforms.Resize((28, 28)),
                        transforms.RandomRotation((-11., 11.), fill=0),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

# Testing data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 205874369.24it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 114820562.87it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 68032299.09it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 7021942.04it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [15]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []


**Light Model**:

***Target***: Building a simple model without any batch normalization, dropout, and global average pooling.

***Results***: Parameters = 8.6K Train Accuracy = 97.01% Test Accuracy = 98.02%

***Analysis***: Model might underperform compared to other versions due to the absence of BN, dropout and GAP also the numbers of paramters have dropped.

In [13]:

# Check if CUDA is available and set PyTorch to use GPU or CPU accordingly
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Initialize the model and send it to GPU if available
model = Net(version=1).to(device)

# Print the model summary
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

torch.manual_seed(1)
batch_size = 512


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
            Conv2d-3           [-1, 10, 24, 24]             730
              ReLU-4           [-1, 10, 24, 24]               0
            Conv2d-5           [-1, 10, 22, 22]             910
              ReLU-6           [-1, 10, 22, 22]               0
         MaxPool2d-7           [-1, 10, 11, 11]               0
            Conv2d-8           [-1, 10, 11, 11]             110
              ReLU-9           [-1, 10, 11, 11]               0
           Conv2d-10             [-1, 10, 9, 9]             910
             ReLU-11             [-1, 10, 9, 9]               0
           Conv2d-12             [-1, 10, 7, 7]             910
             ReLU-13             [-1, 10, 7, 7]               0
           Conv2d-14             [-1, 1

In [16]:
# Define the optimizer
from torch.optim.lr_scheduler import StepLR
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

# Train and test the model
for epoch in range(1, 16):
    print("Epoch: ", epoch)
    print("Training...")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch:  1
Training...


Loss=2.3029563426971436 Batch_id=117 Accuracy=11.24: 100%|██████████| 118/118 [00:27<00:00,  4.23it/s]



Test set: Average loss: 2.3011, Accuracy: 1135/10000 (11.35%)

Epoch:  2
Training...


Loss=2.2906429767608643 Batch_id=117 Accuracy=11.24: 100%|██████████| 118/118 [00:28<00:00,  4.16it/s]



Test set: Average loss: 2.3003, Accuracy: 1135/10000 (11.35%)

Epoch:  3
Training...


...


Test set: Average loss: 0.0570, Accuracy: 9822/10000 (98.22%)

Epoch:  14
Training...


Loss=0.10261467099189758 Batch_id=117 Accuracy=97.03: 100%|██████████| 118/118 [00:28<00:00,  4.20it/s]



Test set: Average loss: 0.0595, Accuracy: 9816/10000 (98.16%)

Epoch:  15
Training...


Loss=0.1254449337720871 Batch_id=117 Accuracy=97.07: 100%|██████████| 118/118 [00:28<00:00,  4.13it/s]



Test set: Average loss: 0.0561, Accuracy: 9826/10000 (98.26%)



**With Batch Normalization (BN)**:

***Target***: Building a model with BN layers , added after each convolution layer to standardize feature maps. As Batch normalization is also to speed up learning and improve generalization

***Results***: Parameters = 8.6K Train Accuracy = 98.76% Test Accuracy = 99.11%

***Analysis***: Batch normalization helped the model train faster and generalize better also the accuracy was improved.

In [17]:

# Check if CUDA is available and set PyTorch to use GPU or CPU accordingly
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Initialize the model and send it to GPU if available
model = Net(version=2).to(device)

# Print the model summary
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

torch.manual_seed(1)
batch_size = 512


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
              ReLU-3            [-1, 8, 26, 26]               0
            Conv2d-4           [-1, 10, 24, 24]             730
       BatchNorm2d-5           [-1, 10, 24, 24]              20
              ReLU-6           [-1, 10, 24, 24]               0
            Conv2d-7           [-1, 10, 22, 22]             910
       BatchNorm2d-8           [-1, 10, 22, 22]              20
              ReLU-9           [-1, 10, 22, 22]               0
        MaxPool2d-10           [-1, 10, 11, 11]               0
           Conv2d-11           [-1, 10, 11, 11]             110
      BatchNorm2d-12           [-1, 10, 11, 11]              20
             ReLU-13           [-1, 10, 11, 11]               0
           Conv2d-14             [-1, 1

In [18]:
# Define the optimizer
from torch.optim.lr_scheduler import StepLR
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

# Train and test the model
for epoch in range(1, 16):
    print("Epoch: ", epoch)
    print("Training...")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch:  1
Training...


Loss=0.13477399945259094 Batch_id=117 Accuracy=83.88: 100%|██████████| 118/118 [00:28<00:00,  4.19it/s]



Test set: Average loss: 0.1201, Accuracy: 9678/10000 (96.78%)

Epoch:  2
Training...


Loss=0.05594182386994362 Batch_id=117 Accuracy=96.10: 100%|██████████| 118/118 [00:28<00:00,  4.18it/s]



Test set: Average loss: 0.0750, Accuracy: 9780/10000 (97.80%)

Epoch:  3
Training...


...


Test set: Average loss: 0.0340, Accuracy: 9878/10000 (98.78%)

Epoch:  14
Training...


Loss=0.007993071340024471 Batch_id=117 Accuracy=98.79: 100%|██████████| 118/118 [00:28<00:00,  4.09it/s]



Test set: Average loss: 0.0299, Accuracy: 9903/10000 (99.03%)

Epoch:  15
Training...


Loss=0.02556598000228405 Batch_id=117 Accuracy=98.76: 100%|██████████| 118/118 [00:28<00:00,  4.17it/s]



Test set: Average loss: 0.0278, Accuracy: 9911/10000 (99.11%)



**With Batch Normalization (BN) and Dropout**:

***Target***: Building a model with BN layers and DP layer , added after each convolution layer. Applying dropout for regularization and preventing overfitting.

***Results***: Parameters = 8.6K Train Accuracy = 97.85% Test Accuracy = 99.10%

***Analysis***: Dropout can help in managing overfitting by providing a form of regularization , the Training Accucracy dipped as expected and The test accuracy remained constant as per the prev model.

In [19]:

# Check if CUDA is available and set PyTorch to use GPU or CPU accordingly
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Initialize the model and send it to GPU if available
model = Net(version=3).to(device)

# Print the model summary
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

torch.manual_seed(1)
batch_size = 512


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
              ReLU-3            [-1, 8, 26, 26]               0
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 10, 24, 24]             730
       BatchNorm2d-6           [-1, 10, 24, 24]              20
              ReLU-7           [-1, 10, 24, 24]               0
           Dropout-8           [-1, 10, 24, 24]               0
            Conv2d-9           [-1, 10, 22, 22]             910
      BatchNorm2d-10           [-1, 10, 22, 22]              20
             ReLU-11           [-1, 10, 22, 22]               0
          Dropout-12           [-1, 10, 22, 22]               0
        MaxPool2d-13           [-1, 10, 11, 11]               0
           Conv2d-14           [-1, 10,

In [20]:
# Define the optimizer
from torch.optim.lr_scheduler import StepLR
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

# Train and test the model
for epoch in range(1, 16):
    print("Epoch: ", epoch)
    print("Training...")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch:  1
Training...


Loss=0.19920028746128082 Batch_id=117 Accuracy=76.77: 100%|██████████| 118/118 [00:29<00:00,  4.01it/s]



Test set: Average loss: 0.1647, Accuracy: 9524/10000 (95.24%)

Epoch:  2
Training...


Loss=0.09923535585403442 Batch_id=117 Accuracy=93.72: 100%|██████████| 118/118 [00:28<00:00,  4.17it/s]



Test set: Average loss: 0.0870, Accuracy: 9754/10000 (97.54%)

Epoch:  3
Training...


...


Test set: Average loss: 0.0307, Accuracy: 9908/10000 (99.08%)

Epoch:  14
Training...


Loss=0.04329134523868561 Batch_id=117 Accuracy=97.70: 100%|██████████| 118/118 [00:31<00:00,  3.78it/s]



Test set: Average loss: 0.0355, Accuracy: 9889/10000 (98.89%)

Epoch:  15
Training...


Loss=0.056093569844961166 Batch_id=117 Accuracy=97.85: 100%|██████████| 118/118 [00:28<00:00,  4.14it/s]



Test set: Average loss: 0.0304, Accuracy: 9910/10000 (99.10%)



**With Batch Normalization (BN) and Dropout and GAP**:

***Target***: Create a model with batch normalization, dropout for regularization and global average pooling for better performance.

***Results***: Parameters = 6.7K Train Accuracy = 98.10% Test Accuracy = 98.16%

***Analysis***:This model was expected to perform best amongst all due to the combination of BN, dropout, and GAP. However, it might also be prone to overfitting due to its complexity, the results were not improved so lets try different LR values.

In [21]:

# Check if CUDA is available and set PyTorch to use GPU or CPU accordingly
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Initialize the model and send it to GPU if available
model = Net(version=4).to(device)

# Print the model summary
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

torch.manual_seed(1)
batch_size = 512


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 10, 24, 24]             720
              ReLU-6           [-1, 10, 24, 24]               0
       BatchNorm2d-7           [-1, 10, 24, 24]              20
           Dropout-8           [-1, 10, 24, 24]               0
            Conv2d-9           [-1, 12, 24, 24]             120
        MaxPool2d-10           [-1, 12, 12, 12]               0
           Conv2d-11           [-1, 14, 10, 10]           1,512
             ReLU-12           [-1, 14, 10, 10]               0
      BatchNorm2d-13           [-1, 14, 10, 10]              28
          Dropout-14           [-1, 14,

In [22]:
# Define the optimizer
from torch.optim.lr_scheduler import StepLR
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

# Train and test the model
for epoch in range(1, 16):
    print("Epoch: ", epoch)
    print("Training...")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch:  1
Training...


Loss=0.9134268760681152 Batch_id=117 Accuracy=50.09: 100%|██████████| 118/118 [00:29<00:00,  3.99it/s]



Test set: Average loss: 0.9530, Accuracy: 7044/10000 (70.44%)

Epoch:  2
Training...


Loss=0.3257836103439331 Batch_id=117 Accuracy=87.23: 100%|██████████| 118/118 [00:29<00:00,  4.04it/s]



Test set: Average loss: 0.2414, Accuracy: 9488/10000 (94.88%)

Epoch:  3
Training...


...


Test set: Average loss: 0.0341, Accuracy: 9894/10000 (98.94%)

Epoch:  14
Training...


Loss=0.06390424817800522 Batch_id=117 Accuracy=98.02: 100%|██████████| 118/118 [00:29<00:00,  4.07it/s]



Test set: Average loss: 0.0327, Accuracy: 9892/10000 (98.92%)

Epoch:  15
Training...


Loss=0.15066666901111603 Batch_id=117 Accuracy=98.10: 100%|██████████| 118/118 [00:28<00:00,  4.08it/s]



Test set: Average loss: 0.0321, Accuracy: 9896/10000 (98.96%)



## LR Experimentation for Final Model 4

In [37]:
# Define the optimizer
from torch.optim.lr_scheduler import StepLR
optimizer = optim.SGD(model.parameters(), lr=0.025, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

# Train and test the model
for epoch in range(1, 16):
    print("Epoch: ", epoch)
    print("Training...")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch:  1
Training...


Loss=0.044084664434194565 Batch_id=117 Accuracy=98.72: 100%|██████████| 118/118 [00:31<00:00,  3.74it/s]



Test set: Average loss: 0.0232, Accuracy: 9930/10000 (99.30%)

Epoch:  2
Training...


Loss=0.012225211597979069 Batch_id=117 Accuracy=98.71: 100%|██████████| 118/118 [00:31<00:00,  3.73it/s]



Test set: Average loss: 0.0214, Accuracy: 9938/10000 (99.38%)

Epoch:  3
Training...


...


Test set: Average loss: 0.0187, Accuracy: 9937/10000 (99.37%)

Epoch:  14
Training...


Loss=0.008282456547021866 Batch_id=117 Accuracy=98.83: 100%|██████████| 118/118 [00:31<00:00,  3.75it/s]



Test set: Average loss: 0.0204, Accuracy: 9934/10000 (99.34%)

Epoch:  15
Training...


Loss=0.11240572482347488 Batch_id=117 Accuracy=98.76: 100%|██████████| 118/118 [00:32<00:00,  3.69it/s]



Test set: Average loss: 0.0207, Accuracy: 9935/10000 (99.35%)



In [38]:
# Define the optimizer
from torch.optim.lr_scheduler import StepLR
optimizer = optim.SGD(model.parameters(), lr=0.023, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

# Train and test the model
for epoch in range(1, 16):
    print("Epoch: ", epoch)
    print("Training...")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch:  1
Training...


Loss=0.0869583860039711 Batch_id=117 Accuracy=98.81: 100%|██████████| 118/118 [00:32<00:00,  3.65it/s]



Test set: Average loss: 0.0207, Accuracy: 9935/10000 (99.35%)

Epoch:  2
Training...


Loss=0.022402873262763023 Batch_id=117 Accuracy=98.82: 100%|██████████| 118/118 [00:31<00:00,  3.78it/s]



Test set: Average loss: 0.0209, Accuracy: 9943/10000 (99.43%)

Epoch:  3
Training...


...


Test set: Average loss: 0.0206, Accuracy: 9938/10000 (99.38%)

Epoch:  14
Training...


Loss=0.01686198078095913 Batch_id=117 Accuracy=98.87: 100%|██████████| 118/118 [00:30<00:00,  3.87it/s]



Test set: Average loss: 0.0220, Accuracy: 9933/10000 (99.33%)

Epoch:  15
Training...


Loss=0.07558518648147583 Batch_id=117 Accuracy=98.95: 100%|██████████| 118/118 [00:31<00:00,  3.73it/s]



Test set: Average loss: 0.0194, Accuracy: 9937/10000 (99.37%)



In [39]:
# Define the optimizer
from torch.optim.lr_scheduler import StepLR
optimizer = optim.SGD(model.parameters(), lr=0.022, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

# Train and test the model
for epoch in range(1, 16):
    print("Epoch: ", epoch)
    print("Training...")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch:  1
Training...


Loss=0.03125995397567749 Batch_id=117 Accuracy=98.92: 100%|██████████| 118/118 [00:31<00:00,  3.79it/s]



Test set: Average loss: 0.0222, Accuracy: 9933/10000 (99.33%)

Epoch:  2
Training...


Loss=0.01921103335916996 Batch_id=117 Accuracy=98.89: 100%|██████████| 118/118 [00:30<00:00,  3.85it/s]



Test set: Average loss: 0.0199, Accuracy: 9938/10000 (99.38%)

Epoch:  3
Training...


...


Test set: Average loss: 0.0185, Accuracy: 9940/10000 (99.40%)

Epoch:  14
Training...


Loss=0.0371144562959671 Batch_id=117 Accuracy=98.91: 100%|██████████| 118/118 [00:31<00:00,  3.73it/s]



Test set: Average loss: 0.0191, Accuracy: 9938/10000 (99.38%)

Epoch:  15
Training...


Loss=0.00762234628200531 Batch_id=117 Accuracy=98.99: 100%|██████████| 118/118 [00:30<00:00,  3.85it/s]



Test set: Average loss: 0.0203, Accuracy: 9938/10000 (99.38%)



## Final Model getting consistent 99.4% accuracy in last few epochs. With LR = 0.021

In [40]:
# Define the optimizer
from torch.optim.lr_scheduler import StepLR
optimizer = optim.SGD(model.parameters(), lr=0.021, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

# Train and test the model
for epoch in range(1, 16):
    print("Epoch: ", epoch)
    print("Training...")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch:  1
Training...


Loss=0.04310225322842598 Batch_id=117 Accuracy=98.95: 100%|██████████| 118/118 [00:30<00:00,  3.92it/s]



Test set: Average loss: 0.0210, Accuracy: 9937/10000 (99.37%)

Epoch:  2
Training...


Loss=0.008080837316811085 Batch_id=117 Accuracy=98.94: 100%|██████████| 118/118 [00:31<00:00,  3.75it/s]



Test set: Average loss: 0.0194, Accuracy: 9943/10000 (99.43%)

Epoch:  3
Training...


...


Test set: Average loss: 0.0162, Accuracy: 9943/10000 (99.43%)

Epoch:  14
Training...


Loss=0.012581090442836285 Batch_id=117 Accuracy=98.97: 100%|██████████| 118/118 [00:31<00:00,  3.74it/s]



Test set: Average loss: 0.0190, Accuracy: 9941/10000 (99.41%)

Epoch:  15
Training...


Loss=0.02066531777381897 Batch_id=117 Accuracy=98.98: 100%|██████████| 118/118 [00:30<00:00,  3.90it/s]



Test set: Average loss: 0.0173, Accuracy: 9944/10000 (99.44%)

